# Function Generation for the Training of λ-Nets

## Package installation (uncommand first line to install packages at the beginning)

In [1]:
%%script false --no-raise-error
#if some errors occur during the installation consider using "sudo pip install"
!pip install numpy
!pip install pandas
!pip install truth-table-generator
!pip install more-itertools
!pip install tqdm
!pip install joblib
!pip install scipy
!pip install PrettyTable
!pip install colored
!pip install scikit-learn
!pip install keras
!pip install ipython
!pip install livelossplot
!pip install matplotlib
!pip install seaborn
!pip install tensorflow
!pip install tensorflow-gpu

## Specitication of Experiment Settings

In [2]:
import math
def nCr(n,r):
    f = math.factorial
    return f(n) // f(r) // f(n-r)

In [3]:
d = 3  
n = 5
sparsity = nCr(n+d, d)


x_max = 1 
x_min = -1
x_step = 0.01
a_max = 10 
a_min = -10
a_step = 0.001

n_jobs = -3

lambda_dataset_size = 1000 #specify the number of data points to calculate the function values for (determines the lambda net training size)

interpretation_dataset_size = 100000 #specifies the number of functions generated (specifies the interpretation-net dataset size)

same_training_all_lambda_nets = False


In [4]:
##############DO NOT CHANGE###################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]

RANDOM_SEED = 42

if same_training_all_lambda_nets:
    training_string = '_same'
else:
    training_string = '_diverse'
    
print('Variables: ' + str(n) + ' (' + variables + ')')
print('Degree: ' + str(d))
print('Sparsity: ' + str(sparsity)) 
print('Lambda-Net Dataset Size: ' + str(lambda_dataset_size))
print('I-Net Dataset Size: ' + str(interpretation_dataset_size))
      
print('Coefficient Range: ' + '[' + str(a_min) + ', ' + str(a_max) + ']')
print('Variable Range: ' + '[' + str(x_min) + ', ' + str(x_max) + ']')

Variables: 5 (abcde)
Degree: 3
Sparsity: 56
Lambda-Net Dataset Size: 1000
I-Net Dataset Size: 100000
Coefficient Range: [-10, 10]
Variable Range: [-1, 1]


## Imports

In [5]:
import ttg
from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 
random.seed(42)

import os

directory_names = ['parameters', 'plotting', 'saved_polynomial_lists', 'results', 'saved_models', 'weights', 'weights_training']
if not os.path.exists('./data'):
    os.mkdir('./data')
for directory_name in directory_names:
    path = './data/' + directory_name
    if not os.path.exists(path):
        os.mkdir(path)
        
import seaborn as sns
sns.set_style("darkgrid")
        
import random 
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)




## Utility functions

In [6]:
#test for exact equality
def arreq_in_list(myarr, list_arrays):
    return next((True for elem in list_arrays if np.array_equal(elem, myarr)), False)

In [7]:
ALPHABET = \
  "0123456789abcdefghijklmnopqrstuvwxyz"

def encode (n):
    try:
        return ALPHABET [n]
    except IndexError:
        raise Exception ("cannot encode: %s" % n)

def dec_to_base (dec = 0, base = 16):
    if dec < base:
        return encode (dec)
    else:
        return dec_to_base (dec // base, base) + encode (dec % base)

# Function Generation

In [8]:
list_of_monomial_identifiers_extended = []
for i in tqdm(range((d+1)**n)):    
    monomial_identifier = dec_to_base(i, base = (d+1)).zfill(n) 
    list_of_monomial_identifiers_extended.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers_extended)

list_of_monomial_identifiers = []
for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
    monomial_identifier_values = list(map(int, list(monomial_identifier)))
    if sum(monomial_identifier_values) <= d:
        list_of_monomial_identifiers.append(monomial_identifier)

print('List length: ' + str(len(list_of_monomial_identifiers)))
print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(nCr(n+d, d)))
print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)



List length: 1024
Number of monomials in a polynomial with 5 variables and degree 3: 56
Sparsity: 56
['00000', '00001', '00002', '00003', '00010', '00011', '00012', '00013', '00020', '00021', '00022', '00023', '00030', '00031', '00032', '00033', '00100', '00101', '00102', '00103', '00110', '00111', '00112', '00113', '00120', '00121', '00122', '00123', '00130', '00131', '00132', '00133', '00200', '00201', '00202', '00203', '00210', '00211', '00212', '00213', '00220', '00221', '00222', '00223', '00230', '00231', '00232', '00233', '00300', '00301', '00302', '00303', '00310', '00311', '00312', '00313', '00320', '00321', '00322', '00323', '00330', '00331', '00332', '00333', '01000', '01001', '01002', '01003', '01010', '01011', '01012', '01013', '01020', '01021', '01022', '01023', '01030', '01031', '01032', '01033', '01100', '01101', '01102', '01103', '01110', '01111', '01112', '01113', '01120', '01121', '01122', '01123', '01130', '01131', '01132', '01133', '01200', '01201', '01202', '01203


List length: 56
Number of monomials in a polynomial with 5 variables and degree 3: 56
Sparsity: 56
['00000', '00001', '00002', '00003', '00010', '00011', '00012', '00020', '00021', '00030', '00100', '00101', '00102', '00110', '00111', '00120', '00200', '00201', '00210', '00300', '01000', '01001', '01002', '01010', '01011', '01020', '01100', '01101', '01110', '01200', '02000', '02001', '02010', '02100', '03000', '10000', '10001', '10002', '10010', '10011', '10020', '10100', '10101', '10110', '10200', '11000', '11001', '11010', '11100', '12000', '20000', '20001', '20010', '20100', '21000', '30000']


In [9]:
list_of_polynomials = []

if interpretation_dataset_size/((a_max-a_min)*10**int(-np.log10(a_step)))**(nCr(n+d, d)) <= 10e-4:
    while len(list_of_polynomials) < interpretation_dataset_size:
        random_polynomial = list(random_product([i*a_step for i in range(int(a_min*10**int(-np.log10(a_step))), int(a_max*10**int(-np.log10(a_step))))], repeat=nCr(n+d, d)))
        if random_polynomial not in list_of_polynomials:
            list_of_polynomials.append(random_polynomial)
else:
    all_polynomials_list = list(product([i*a_step for i in range(int(a_min*10**int(-np.log10(a_step))), int(a_max*10**int(-np.log10(a_step))))], repeat=nCr(n+d, d)))
    list_of_polynomials = [all_polynomials_list[i] for i in np.random.choice(len(all_polynomials_list), interpretation_dataset_size, replace=False)]
    del all_polynomials_list

for polynomial in tqdm(list_of_polynomials):
    sparsity_indices = np.random.choice(nCr(n+d, d), nCr(n+d, d)-sparsity, replace=False)
    for sparsity_index in sparsity_indices:                            
        polynomial[sparsity_index] = 0
                                
polynomials_list_df = pd.DataFrame(data=list_of_polynomials, columns=list_of_monomial_identifiers)

    
print(len(list_of_monomial_identifiers))
print(polynomials_list_df.shape)


56
(100000, 56)


In [10]:
polynomials_list_df.head()

,00000,00001,00002,00003,00010,00011,00012,00020,00021,00030,...,11001,11010,11100,12000,20000,20001,20010,20100,21000,30000
0,-6.352,-9.181,-0.988,-1.976,-2.686,-5.428,-6.642,7.870,-7.152,9.349,...,-5.910,2.403,-7.418,8.089,-0.394,1.850,8.918,-3.700,-7.721,-8.499
1,-2.533,-0.518,-7.386,-2.372,-6.691,2.455,-0.892,4.857,1.954,-4.671,...,-1.391,9.155,4.038,9.121,3.087,1.861,-2.814,-5.468,6.696,6.171
2,-7.022,-8.457,-6.408,-4.992,-4.758,3.833,9.543,-7.919,2.608,2.504,...,1.894,0.076,-2.154,-8.102,-2.108,8.591,-7.420,-7.194,5.924,-7.733
3,7.455,-5.880,-5.793,5.574,8.015,-4.590,-1.315,7.290,9.876,3.865,...,8.881,5.488,-2.038,5.498,3.338,-3.761,-6.910,-6.824,4.124,1.609
4,3.879,3.470,5.303,-8.225,-6.776,-8.014,3.193,1.118,-6.420,-1.853,...,-4.928,-3.778,-0.278,-2.867,-8.084,8.978,7.766,-8.003,0.276,-8.127


In [11]:
def generate_random_x_values(size, x_max, x_min, x_step, numnber_of_variables):
    x_values_list = []
    
    for j in range(size):
        values = np.round(np.array(random_product(np.arange(x_min, x_max, x_step), repeat=numnber_of_variables)), int(-np.log10(x_step)))
        while arreq_in_list(values, x_values_list):
                values = np.round(np.array(random_product(np.arange(x_min, x_max, x_step), repeat=numnber_of_variables)), int(-np.log10(x_step)))         
        x_values_list.append(values)
    
    return x_values_list


In [12]:
if same_training_all_lambda_nets:
    x_values = generate_random_x_values(lambda_dataset_size, x_max, x_min, x_step, n)
    x_values_list = [x_values for _ in range(interpretation_dataset_size)]
else:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    x_values_list = parallel(delayed(generate_random_x_values)(lambda_dataset_size, x_max, x_min, x_step, n) for _ in range(interpretation_dataset_size))
    del parallel


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-3)]: Done 1108 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-3)]: Done 1524 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-3)]: Done 2004 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-3)]: Done 2548 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-3)]: Done 3156 tasks      | elapsed: 18.9min
[Parallel(n_jobs=-3)]: Done 3828 tasks      | elapsed: 22.9min
[Parallel(n_jobs=-3)]: Done 4564 tasks      | elapsed: 27.3min
[Parallel(n_jobs=-3)]: Done 5364 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-3)]: Done 6228 tasks      | elapsed: 37.2min
[Parallel(n_jobs=-3)]: Done 7156 tasks      | elapsed: 42.7min
[Parallel(n_jobs=-3)]: Done 8148 tasks      |

In [13]:
from functools import reduce

def calcualate_function_with_data(coefficient_list, variable_values):
    
    result = 0    
    for coefficient_value, coefficient_multipliers in zip(coefficient_list, list_of_monomial_identifiers):
        partial_results = [variable_value**int(coefficient_multiplier) for coefficient_multiplier, variable_value in zip(coefficient_multipliers, variable_values)]
        
        result += coefficient_value * reduce(lambda x, y: x*y, partial_results)

    return result, variable_values
 
def calculate_function_values_from_polynomial(true_value_test, evaluation_dataset):
    if isinstance(true_value_test, pd.DataFrame):
        true_value_test = true_value_test.values
        
    true_value_fv = []
    true_value_coeff = []
    
    for evaluation in evaluation_dataset:
        true_function_value, true_coeff = calcualate_function_with_data(true_value_test, evaluation)
       
        true_value_fv.append(true_function_value) 
        true_value_coeff.append(true_coeff)
        
    return [true_value_test, pd.DataFrame(np.array(true_value_coeff))], [true_value_test, pd.DataFrame(np.array(true_value_fv))]


In [14]:
result_list = []

polynomials_X_data_list = []
polynomials_y_data_list = []
    
chunks = max(interpretation_dataset_size//1000, 1)

for polynomials_list_df_chunk in tqdm(np.array_split(polynomials_list_df, chunks), total=chunks):
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_sublist = parallel(delayed(calculate_function_values_from_polynomial)(polynomial, x_values) for (_, polynomial), x_values in zip(polynomials_list_df_chunk.iterrows(), x_values_list))  
    result_list.extend(result_sublist)
    del parallel

polynomials_X_data_list = [result[0] for result in result_list]
polynomials_y_data_list = [result[1] for result in result_list]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   51.7s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.2min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.1min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   19.0s
[Pa

[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   53.9s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.1min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   43.1s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   57.3s finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   48.6s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.0min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent w

[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   33.8s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   50.6s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.1min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   43.6s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   57.4s finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-

[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   57.9s finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   33.0s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.1min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-

[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   58.9s finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   57.9s finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   43.5s
[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:  1.2min finished
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed: 

[Parallel(n_jobs=-3)]: Done 1000 out of 1000 | elapsed:   59.2s finished


In [15]:
polynomials_X_data_list[0][0].head()

00000   -6.352
00001   -9.181
00002   -0.988
00003   -1.976
00010   -2.686
Name: 0, dtype: float64

In [16]:
polynomials_X_data_list[0][1].head()

,0,1,2,3,4
0,-0.87,0.40,0.78,-0.47,0.79
1,-0.69,0.61,0.24,0.27,0.29
2,0.12,0.73,-0.72,0.55,0.22
3,0.63,-0.68,-0.04,0.30,0.03
4,0.70,-0.87,-0.67,-0.05,-0.76


In [17]:
polynomials_y_data_list[0][0].values

array([-6.352, -9.181, -0.988, -1.976, -2.686, -5.428, -6.642,  7.87 ,
       -7.152,  9.349,  3.825, -8.959, -9.024, -6.93 , -2.836, -2.377,
        6.559,  9.726, -9.131,  8.39 , -3.485,  7.856,  3.746, -2.777,
        4.719,  9.309, -0.885, -9.788, -4.769,  3.848,  1.149, -0.895,
       -4.906, -2.945,  1.029, -6.651, -6.961,  2.449, -6.831,  1.763,
        1.27 ,  9.782, -1.333, -8.577,  5.054,  7.571, -5.91 ,  2.403,
       -7.418,  8.089, -0.394,  1.85 ,  8.918, -3.7  , -7.721, -8.499])

In [18]:
polynomials_y_data_list[0][0].head()

00000   -6.352
00001   -9.181
00002   -0.988
00003   -1.976
00010   -2.686
Name: 0, dtype: float64

In [19]:
polynomials_y_data_list[0][1].head()

,0
0,-2.997695
1,-8.053974
2,-0.570445
3,-10.924270
4,-3.390441


In [20]:
polynomials_X_data_list[0]

[00000   -6.352
 00001   -9.181
 00002   -0.988
 00003   -1.976
 00010   -2.686
 00011   -5.428
 00012   -6.642
 00020    7.870
 00021   -7.152
 00030    9.349
 00100    3.825
 00101   -8.959
 00102   -9.024
 00110   -6.930
 00111   -2.836
 00120   -2.377
 00200    6.559
 00201    9.726
 00210   -9.131
 00300    8.390
 01000   -3.485
 01001    7.856
 01002    3.746
 01010   -2.777
 01011    4.719
 01020    9.309
 01100   -0.885
 01101   -9.788
 01110   -4.769
 01200    3.848
 02000    1.149
 02001   -0.895
 02010   -4.906
 02100   -2.945
 03000    1.029
 10000   -6.651
 10001   -6.961
 10002    2.449
 10010   -6.831
 10011    1.763
 10020    1.270
 10100    9.782
 10101   -1.333
 10110   -8.577
 10200    5.054
 11000    7.571
 11001   -5.910
 11010    2.403
 11100   -7.418
 12000    8.089
 20000   -0.394
 20001    1.850
 20010    8.918
 20100   -3.700
 21000   -7.721
 30000   -8.499
 Name: 0, dtype: float64,         0     1     2     3     4
 0   -0.87  0.40  0.78 -0.47  0.79
 1   -0.6

In [21]:
polynomials_y_data_list[0]

[00000   -6.352
 00001   -9.181
 00002   -0.988
 00003   -1.976
 00010   -2.686
 00011   -5.428
 00012   -6.642
 00020    7.870
 00021   -7.152
 00030    9.349
 00100    3.825
 00101   -8.959
 00102   -9.024
 00110   -6.930
 00111   -2.836
 00120   -2.377
 00200    6.559
 00201    9.726
 00210   -9.131
 00300    8.390
 01000   -3.485
 01001    7.856
 01002    3.746
 01010   -2.777
 01011    4.719
 01020    9.309
 01100   -0.885
 01101   -9.788
 01110   -4.769
 01200    3.848
 02000    1.149
 02001   -0.895
 02010   -4.906
 02100   -2.945
 03000    1.029
 10000   -6.651
 10001   -6.961
 10002    2.449
 10010   -6.831
 10011    1.763
 10020    1.270
 10100    9.782
 10101   -1.333
 10110   -8.577
 10200    5.054
 11000    7.571
 11001   -5.910
 11010    2.403
 11100   -7.418
 12000    8.089
 20000   -0.394
 20001    1.850
 20010    8.918
 20100   -3.700
 21000   -7.721
 30000   -8.499
 Name: 0, dtype: float64,              0
 0    -2.997695
 1    -8.053974
 2    -0.570445
 3   -10.924270

In [22]:
path_polynomials = './data/saved_polynomial_lists/polynomials_sample' + str(interpretation_dataset_size) + '_variables_' + str(n) +  '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step)  + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '.csv'
polynomials_list_df.to_csv(path_polynomials, index=False)

path_X_data = './data/saved_polynomial_lists/X_sample' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(polynomials_X_data_list, f)#, protocol=2)
    
path_y_data = './data/saved_polynomial_lists/y_sample' + str(interpretation_dataset_size) + '_train_' + str(lambda_dataset_size) + '_variables_' + str(n) + '_degree_' + str(d) + '_sparsity_' + str(sparsity) + '_astep_' + str(a_step) + '_amin_' + str(a_min) + '_amax_' + str(a_max) + '_xstep_' + str(x_step) + '_xmin_' + str(x_min) + '_xmax_' + str(x_max) + training_string + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(polynomials_y_data_list, f)#, protocol=2)
